# Tracing

Documentation:

https://docs.smith.langchain.com/tutorials/Developers/observability

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.environ.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="langsmith-basics"

In [23]:
from langchain_openai import ChatOpenAI
from langsmith import traceable


llm = ChatOpenAI(model="gpt-4o-mini")

# This is the retriever we will use in RAG
# This is mocked out, but it could be anything we want

@traceable(run_type="retriever")
def retriever(query: str):
    results = ["Harrison worked at Kensho"]
    return results

# This is the end-to-end RAG chain.
# It does a retrieval step then calls OpenAI
@traceable
def rag(question):
    docs = retriever(question)
    system_message = """Answer the users question using only the provided information below:
    
    {docs}""".format(docs="\n".join(docs))
    
    return llm(
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ],
        model="gpt-4o-mini",
    )

In [26]:
retriever("Where did Harrison work?")

['Harrison worked at Kensho']

In [25]:
rag("Where did Harrison work?")

AIMessage(content='Harrison worked at Kensho.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 34, 'total_tokens': 41}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-2c57556d-76e2-4fee-8e7e-2e3e7fc166a7-0', usage_metadata={'input_tokens': 34, 'output_tokens': 7, 'total_tokens': 41})